In [2]:
import sys
import numpy as np
np.random.seed(42)
import gensim
import keras.backend as K
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Embedding, Lambda
from keras.utils import np_utils
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer

In [5]:
# In case your sys.path does not contain the base repo, cd there.
print(sys.path)
%cd 'PATH_OF_BASE_REPO'  # In the solution it will be the path to my repo. This is such that python loads al the files from the top.

['/Users/axelsirota/repos/ml-solr-course/1-synonyms/lab1/exercise', '/Users/axelsirota/repos/ml-solr-course', '/Users/axelsirota/.pyenv/versions/3.7.3/lib/python37.zip', '/Users/axelsirota/.pyenv/versions/3.7.3/lib/python3.7', '/Users/axelsirota/.pyenv/versions/3.7.3/lib/python3.7/lib-dynload', '', '/Users/axelsirota/repos/ml-solr-course/.venv/lib/python3.7/site-packages', '/Users/axelsirota/repos/ml-solr-course/.venv/lib/python3.7/site-packages/IPython/extensions', '/Users/axelsirota/.ipython', '/Users/axelsirota/Library/Application Support/JetBrains/Toolbox/apps/PyCharm-P/ch-0/211.7442.45/PyCharm.app/Contents/plugins/python/helpers/pydev', '/Users/axelsirota/Library/Application Support/JetBrains/Toolbox/apps/PyCharm-P/ch-0/211.7442.45/PyCharm.app/Contents/plugins/python/helpers-pro/jupyter_debug']
[Errno 2] No such file or directory: 'PATH_OF_BASE_REPO # In the solution it will be the path to my repo. This is such that python loads al the files from the top.'
/Users/axelsirota/repos/

In [6]:
# First let's load the dataset
path = 'dataset/docv2_train_queries.tsv'
queries = pd.read_csv(path, sep='\t', lineterminator='\r', names=['query_id', 'query'])
queries.head()


/Users/axelsirota/repos/ml-solr-course/.venv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,query_id,query
0,121352,define extreme
1,510633,tattoo fixers how much does it cost
2,674172,what is a bank transit number
3,570009,what are the four major groups of elements
4,54528,blood clots in urine after menopause


In [7]:
corpus = [sentence for sentence in queries['query'].values if type(sentence) == str and len(sentence.split(' ')) >= 3]


At this point we have a list (any iterable will do) of queries that are longer than 3 words. This is normal to filter random queries. Now we must use the `Tokenizer` object to `fit` on the corpus, in order to convert each wor to an ID, and later convert such corpus of list of words into their identifiers.


In [1]:
tokenizer = Tokenizer()
# Use the fit_on_text method to fit the tokenizer
None # Fill

print(f'Before the tokenizer: {corpus[:1]}')

#Now use the same "trained" tokenizer to convert the corpus from words to IDs with the text_to_sequences method
corpus = None

print(f'After the tokenizer: {corpus[:1]}')



NameError: name 'Tokenizer' is not defined

In [ ]:
nb_samples = sum(len(s) for s in corpus)
V = len(tokenizer.word_index) + 1   # Size of the vocabulary, adding the UNK word.
dim = 100  # size of the embedding to create
window_size = 3 
epochs=50
batch_size = 1000  # Note that this is because the workstation doesn't have a GPU. In reallity we would like a batch size and events such that we passthrough the dataset a couple of times
BATCH = True


In [ ]:
print(f'First 5 corpus items are {corpus[:5]}')
print(f'Length of corpus is {len(corpus)}')


Now comes the core part, defining the model. Keras provides a convenient Sequential model class to just `add` layers of any type and they will just work. Let's add an `Embedding` layer (that will map the word ids into a vector of size 100), a `Lambda` to average the words out in a sentence, and a `Dense layer` to select the best word on the other end. This is classic CBOW. 

In [ ]:
cbow = Sequential()
cbow.add()  # Add an Embedding layer with input_dim V, output_dim to be 100, and the input_length to be twice our window
cbow.add()  # Add a Lambda that takes a lambda function using the K.mean method to average the words. The output_shape should be (dim, ).
cbow.add(Dense(V, activation='softmax'))  # We add a classic Dense layer to just select with a softmax the best word

In [ ]:
# Compile the model with a loss and optimizer of your liking.
cbow.compile()
cbow.summary()

In [ ]:
# This is the algorithmic part of batching the dataset and yielding the window of words and expected middle word for each bacth as a generator.
def generate_data(corpus, window_size, V, batch_size=batch_size):
    number_of_batches = (len(corpus) // batch_size) + 1
    for batch in range(number_of_batches):
        lower_end = batch*batch_size
        upper_end = (batch+1)*batch_size if batch+1 < number_of_batches else len(corpus)
        mini_batch_size = upper_end - lower_end
        maxlen = window_size*2
        X = np.zeros((mini_batch_size, maxlen))
        Y = np.zeros((mini_batch_size, V))
        for query_id, words in enumerate(corpus[lower_end:upper_end]):
            L = len(words)
            for index, word in enumerate(words):
                contexts = []
                labels   = []            
                s = index - window_size
                e = index + window_size + 1

                contexts.append([words[i] for i in range(s, e) if 0 <= i < L and i != index])
                labels.append(word)

                x = sequence.pad_sequences(contexts, maxlen=maxlen)
                y = np_utils.to_categorical(labels, V)
                X[query_id] = x
                Y[query_id] = y
        yield (X, Y)



In [ ]:
# If data is small, you can just generate the whole dataset and load it in memory to use the fit method
#
# def generate_data(corpus, window_size, V):
#         maxlen = window_size*2
#         X = np.zeros((len(corpus), maxlen))
#         Y = np.zeros((len(corpus), V))
#         for query_id, words in enumerate(corpus):
#             L = len(words)
#             for index, word in enumerate(words):
#                 contexts = []
#                 labels   = []            
#                 s = index - window_size
#                 e = index + window_size + 1

#                 contexts.append([words[i] for i in range(s, e) if 0 <= i < L and i != index])
#                 labels.append(word)

#                 x = sequence.pad_sequences(contexts, maxlen=maxlen)
#                 y = np_utils.to_categorical(labels, V)
#                 X[query_id] = x
#                 Y[query_id] = y
#         return (X, Y)


In [ ]:
def fit_model():
    if not BATCH:
        X, Y = generate_data(corpus, window_size, V)
        print(f'Size of X is {X.shape} and Y is {Y.shape}')
        cbow.fit(X, Y, epochs = epochs)
    else:
        index = 1
        for x, y in generate_data(corpus, window_size, V):
            print(f'Training on Iteration: {index}')
            index += 1
            history = cbow.train_on_batch(x, y, reset_metrics=False, return_dict=True)
            print(history)
            if index > epochs:
                break

In [ ]:
fit_model()

What happens with the losses? Try to plot them to see what trend the follow! Could it happen that some iteration gets a bigger loss than a previous iteration?

In [ ]:
# For each word we save the weights in a standard format
with open('./1-synonyms/lab1/vectors.txt' ,'w') as f:
    f.write('{} {}\n'.format(V-1, dim))
    vectors = cbow.get_weights()[0]
    for word, i in tokenizer.word_index.items():
        str_vec = ' '.join(map(str, list(vectors[i, :])))
        f.write('{} {}\n'.format(word, str_vec))


In [ ]:
w2v = gensim.models.KeyedVectors.load_word2vec_format('../1-synonyms/lab1/vectors.txt', binary=False)

Check if they make sense. You can play a bit.

In [ ]:
w2v.most_similar(positive=['gasoline'])

In [ ]:
w2v.most_similar(negative=['apple'])